In [ ]:
from src.models.cyclegan import CycleGan
import pytorch_lightning as pl
from src.datamodule import CelebAData
pl.seed_everything()

In [ ]:
import os
import pandas as pd
root = './data/celeba'
attributes = pd.read_csv(os.path.join(root, 'list_attr_celeba.csv'))
images_dir = os.path.join(root, 'img_align_celeba')


In [ ]:
good_attribute_threshold = 0.3
good_attributes = ['image_id']
for key in attributes.keys():
    c = attributes[attributes[key]==1]['image_id'].count()
    if c < len(attributes)*(1-good_attribute_threshold) and c > len(attributes) * good_attribute_threshold:
        good_attributes.append(key)
gdf = attributes[good_attributes]

In [ ]:
list(attributes.keys()).index("Mouth_Slightly_Open")
from argparse import Namespace
print(good_attributes)
print(list(attributes.keys()).index("Attractive"))

In [ ]:
args = Namespace(
    log_every_n_steps=1,
    gradient_clip_val=0.5,
    learning_rate=1e-3,
    weight_decay=1e-6,
    precision=16,
    data_dir = './data/celeba',
    batch_size = 512,
    num_workers = 1,
    gpus=2,
    max_epochs=10,
    accumulate_grad_batches=4,
)

In [ ]:
from src.models.classifier import Classifier, Identity
from src.models.cnn_for_encoded import SimpleCNN
from torchsummary import summary
cnn = SimpleCNN().to('cuda:0')
target_attr = 32
# translator = Identity()
translator = CycleGan(args, target_attr=target_attr).load_from_checkpoint('./logs/0126/version_0/checkpoints/epoch=99-step=7999.ckpt').A2B
classifier = Classifier(args, model=cnn, target_attr=target_attr, preEncoder=translator).to('cuda:0')

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger("logs/toy", name='')


In [ ]:
import sys
sys.argv = ['-f']

In [ ]:
import pytorch_lightning as pl
from src.datamodule import CelebAEncodedData
from argparse import ArgumentParser
parser = ArgumentParser()
parser = pl.Trainer.add_argparse_args(parser)
argu = vars(parser.parse_args())
argu.update(vars(args))
args = Namespace(**argu)
trainer = pl.Trainer.from_argparse_args(args, logger=logger)
dm = CelebAEncodedData(args)
trainer.fit(classifier, datamodule=dm)